In [4]:
!pip install psycopg2-binary

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 748.6 kB/s eta 0:00:0000:0100:01


In [ ]:
!pip install pandas 

In [ ]:
from sqlalchemy import create_engine
import pandas as pd
from time import time

In [24]:
def database_connect(user, password, host, port, db):
    engine = create_engine(
        f'postgresql://{user}:{password}@{host}:{port}/{db}')
    engine.connect()
    return engine

def fetch_data(url, csv_name): 
    os.system(f"wget {url} -O {csv_name}")

def batch_load_data(engine, csv_file, table_name): 
    df_iter = pd.read_csv(csv_file, iterator=True, chunksize=100000)
    df = next(df_iter)

    df.head(n=0).to_sql(name=table_name, con=engine, if_exists='replace')
    df.to_sql(name=table_name, con=engine, if_exists='append')

    elapsed_time = -1
    try:
        t_start = time()
        chuncks_cnt = 0
        
        while True:
            chuncks_cnt += 1
            chunck_start = time()
        
            df = next(df_iter)
            df.to_sql(name=table_name, con=engine, if_exists='append')
            print(f'Inserted chunck {chuncks_cnt} in {time()-chunck_start:.3}s')
    except:
        elapsed_time = time() - t_start 
        print(f'Finished ingestion into table: {table_name}')
        print(f'Number of chuncks: {chuncks_cnt}')
        print(f'Time taken: {elapsed_time:.3}s')
        
    return elapsed_time
    

In [35]:
db_engine = database_connect('root', 'root', 'postgres-db', '5432', 'RetailDB')

In [34]:
batch_load_data(db_engine, 'online_retail.csv', 'online_retail')

Inserted chunck 1 in 3.63s
Inserted chunck 2 in 2.94s
Inserted chunck 3 in 3.07s
Inserted chunck 4 in 3.13s
Inserted chunck 5 in 1.22s
Finished ingestion into table: online_retail
Number of chuncks: 6
Time taken: 14.0s


13.997101068496704